In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import norm

# Obtener la lista de rutas de los archivos CSV
dspaths = []
for dirname, _, filenames in os.walk('/TFM/DATASET-2-2019/processed'):
    for filename in filenames:
        if filename.endswith('.csv'):
            pds = os.path.join(dirname, filename)
            dspaths.append(pds)
            print(pds)

# Cargar los archivos CSV en un diccionario de DataFrames
data_frames = {}
for file in dspaths:
    file_name = os.path.basename(file)
    data_frames[file_name] = pd.read_csv(file)


/TFM/DATASET-2-2019/processed\processed_DrDoS_DNS.csv
/TFM/DATASET-2-2019/processed\processed_DrDoS_LDAP.csv
/TFM/DATASET-2-2019/processed\processed_DrDoS_MSSQL.csv
/TFM/DATASET-2-2019/processed\processed_DrDoS_NetBIOS.csv
/TFM/DATASET-2-2019/processed\processed_DrDoS_NTP.csv
/TFM/DATASET-2-2019/processed\processed_DrDoS_SNMP.csv
/TFM/DATASET-2-2019/processed\processed_DrDoS_SSDP.csv
/TFM/DATASET-2-2019/processed\processed_DrDoS_UDP.csv
/TFM/DATASET-2-2019/processed\processed_LDAP.csv
/TFM/DATASET-2-2019/processed\processed_MSSQL.csv
/TFM/DATASET-2-2019/processed\processed_NetBIOS.csv
/TFM/DATASET-2-2019/processed\processed_Portmap.csv
/TFM/DATASET-2-2019/processed\processed_Syn.csv
/TFM/DATASET-2-2019/processed\processed_Syn2.csv
/TFM/DATASET-2-2019/processed\processed_TFTP.csv
/TFM/DATASET-2-2019/processed\processed_UDP.csv
/TFM/DATASET-2-2019/processed\processed_UDPLag.csv
/TFM/DATASET-2-2019/processed\processed_UDPLag2.csv


In [2]:
# Crear un DataFrame con todas las muestras benignas
# Concatenate all benign samples
benign_samples = pd.concat([df[df.iloc[:, 77] == 'BENIGN'] for df in data_frames.values()])
print(f"Label: BENIGN, Number of samples: {len(benign_samples)}")

# Gaussian distribution parameters
mu = 0.5
sigma = 0.2

# Initialize dictionary for malignant samples by label
malign_samples_by_label = {}

# Process each DataFrame
for file, df in data_frames.items():
    # Verificar y eliminar la columna " CWE Flag Count" con espacio al inicio
    if ' CWE Flag Count' in df.columns:
        df.drop(columns=[' CWE Flag Count'], inplace=True)
    
    # Standardize the label column by replacing 'UDP-lag' and 'UDP-Lag' with 'UDPLag'
    df.iloc[:, 76] = df.iloc[:, 76].replace(['UDP-lag', 'UDP-Lag'], 'UDPLag')
    
    # Get only malignant samples and drop of WEBDDOS
    malign_samples = df[(df.iloc[:, 76] != 'BENIGN') & (df.iloc[:, 76] != 'WebDDoS')]
    
    # Group by label and collect samples
    for label in malign_samples.iloc[:, 76].unique():
        if label not in malign_samples_by_label:
            malign_samples_by_label[label] = pd.DataFrame()
        
        malign_samples_by_label[label] = pd.concat([malign_samples_by_label[label], malign_samples[malign_samples.iloc[:, 76] == label]])

# Display the number of malignant samples per label
for label, samples in malign_samples_by_label.items():
    malign_count = len(samples)
    print(f"Label: {label}, Number of malignant samples: {malign_count}")

Label: BENIGN, Number of samples: 113828
Label: DrDoS_DNS, Number of malignant samples: 5071011
Label: DrDoS_LDAP, Number of malignant samples: 2179930
Label: DrDoS_MSSQL, Number of malignant samples: 4522492
Label: DrDoS_NetBIOS, Number of malignant samples: 4093279
Label: DrDoS_NTP, Number of malignant samples: 1202642
Label: DrDoS_SNMP, Number of malignant samples: 5159870
Label: DrDoS_SSDP, Number of malignant samples: 2610611
Label: DrDoS_UDP, Number of malignant samples: 3134645
Label: NetBIOS, Number of malignant samples: 3657497
Label: LDAP, Number of malignant samples: 1915122
Label: MSSQL, Number of malignant samples: 5787453
Label: Portmap, Number of malignant samples: 186960
Label: Syn, Number of malignant samples: 6473789
Label: TFTP, Number of malignant samples: 20082580
Label: UDP, Number of malignant samples: 3867155
Label: UDPLag, Number of malignant samples: 368334


In [3]:
# Función para eliminar la columna " CWE Flag Count"
def remove_cwe_flag_count_column(df):
    # Imprimir las columnas antes de intentar eliminar
    print(f"Columns before drop: {df.columns.tolist()}")
    
    # Verificar y eliminar la columna " CWE Flag Count" con espacio al inicio
    if ' CWE Flag Count' in df.columns:
        df.drop(columns=[' CWE Flag Count'], inplace=True)
    
    # Imprimir las columnas después de intentar eliminar
    print(f"Columns after drop: {df.columns.tolist()}")
    
    return df

#### Realizamos la division de los datos en funcion de la distribuccion Gaussiana

In [4]:
from scipy.stats import norm

# Gaussian distribution parameters
mu = 0.5
sigma = 0.2

# Initialize dictionary for reduced malignant samples by label
malign_reduced_samples_by_label = {}

# Process each DataFrame
for label, samples in malign_samples_by_label.items():
    # Calculate the probability density function of each sample
    samples['prob'] = norm.pdf(np.linspace(0, 1, len(samples)), mu, sigma)
    
    # Reduce samples to 113828 using the probability weights
    malign_reduced_samples_by_label[label] = samples.sample(n=113828, weights='prob', random_state=42)

# Concatenate all reduced malignant samples into one DataFrame
malign_reduced_data = pd.concat(malign_reduced_samples_by_label.values())

# Drop the 'prob' column
malign_reduced_data = malign_reduced_data.drop('prob', axis=1)

# Concatenate all benign samples into one DataFrame
benign_data = benign_samples

# Concatenate all benign and reduced malignant samples into one DataFrame
all_data = pd.concat([benign_data, malign_reduced_data])

# Función para eliminar la columna " CWE Flag Count"
def remove_cwe_flag_count_column(df):
    # Imprimir las columnas antes de intentar eliminar
    print(f"Columns before drop: {df.columns.tolist()}")
    
    # Verificar y eliminar la columna " CWE Flag Count" con espacio al inicio
    if ' CWE Flag Count' in df.columns:
        df.drop(columns=[' CWE Flag Count'], inplace=True)
    
    # Imprimir las columnas después de intentar eliminar
    print(f"Columns after drop: {df.columns.tolist()}")
    
    return df

# Verificar la eliminación de la columna después de la casilla 32
for file, df in data_frames.items():
    df = remove_cwe_flag_count_column(df)
    if ' CWE Flag Count' not in df.columns:
        print(f"Column ' CWE Flag Count' successfully removed in file {file}")
    else:
        print(f"Failed to remove column ' CWE Flag Count' in file {file}")

Columns before drop: [' Protocol', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count'

In [5]:

from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
train_data, test_val_data = train_test_split(all_data, test_size=0.3, random_state=42)

# Eliminar la columna " CWE Flag Count" de los conjuntos de datos de entrenamiento y prueba
if ' CWE Flag Count' in train_data.columns:
    train_data.drop(columns=[' CWE Flag Count'], inplace=True)
if ' CWE Flag Count' in test_val_data.columns:
    test_val_data.drop(columns=[' CWE Flag Count'], inplace=True)


# Dividir el conjunto de prueba en conjuntos de prueba y validación (50% prueba, 50% validación)
test_data, val_data = train_test_split(test_val_data, test_size=0.5, random_state=42)

# Guardar los conjuntos de entrenamiento en archivos CSV
train_data.to_csv('train_2.csv', index=False)

# Guardar los conjuntos de prueba en archivos CSV
test_val_data.to_csv('test_2.csv', index=False)

# Guardar los conjuntos de validación en archivos CSV
val_data.to_csv('validation.csv', index=False)

In [6]:
"""
# Crear un DataFrame con todas las muestras benignas
#benign_samples = pd.concat([df for df in data_frames.values() if df.iloc[:, 77].eq('BENIGN').any()])
benign_samples = pd.concat([df[df.iloc[:, 77].eq('BENIGN')] for df in data_frames.values()])
##benign_samples = benign_samples.sample(n=10000, random_state=42)
print(f"Length: {len(benign_samples)}")
# Reducir las muestras malignas de cada fichero utilizando la función gaussiana para seleccionar las muestras
mu = 0.5  # Media de la distribución gaussiana
sigma = 0.2  # Desviación estándar de la distribución gaussiana

malign_samples_reduced = {}
for file, df in data_frames.items():
    # Obtener solo las muestras malignas (sin etiqueta BENIGN)
    malign_samples = df[df.iloc[:, 77] != 'BENIGN']

    # Calcular las probabilidades gaussianas
    #malign_samples['prob'] = norm.pdf(np.linspace(0, 1, len(malign_samples)), mu, sigma)
    ##malign_samples.loc[:, 'prob'] = norm.pdf(np.linspace(0, 1, len(malign_samples)), mu, sigma)

    # Reducir las muestras malignas a 10000 por fichero
    ##malign_samples_reduced[file] = malign_samples.sample(n=10000, weights='prob', random_state=42)

"""

'\n# Crear un DataFrame con todas las muestras benignas\n#benign_samples = pd.concat([df for df in data_frames.values() if df.iloc[:, 77].eq(\'BENIGN\').any()])\nbenign_samples = pd.concat([df[df.iloc[:, 77].eq(\'BENIGN\')] for df in data_frames.values()])\n##benign_samples = benign_samples.sample(n=10000, random_state=42)\nprint(f"Length: {len(benign_samples)}")\n# Reducir las muestras malignas de cada fichero utilizando la función gaussiana para seleccionar las muestras\nmu = 0.5  # Media de la distribución gaussiana\nsigma = 0.2  # Desviación estándar de la distribución gaussiana\n\nmalign_samples_reduced = {}\nfor file, df in data_frames.items():\n    # Obtener solo las muestras malignas (sin etiqueta BENIGN)\n    malign_samples = df[df.iloc[:, 77] != \'BENIGN\']\n\n    # Calcular las probabilidades gaussianas\n    #malign_samples[\'prob\'] = norm.pdf(np.linspace(0, 1, len(malign_samples)), mu, sigma)\n    ##malign_samples.loc[:, \'prob\'] = norm.pdf(np.linspace(0, 1, len(malign_sa

Creamos los ficheros con datos malignos todos juntos y benigno. Lo cramos una sola vez

In [7]:
"""
# Concatenar todos los DataFrames con datos benignos en un solo DataFrame
benign_data = pd.concat([benign_samples])

# Concatenar todos los DataFrames con datos malignos reducidos en un solo DataFrame
malign_reduced_data = pd.concat(malign_samples_reduced)

# Eliminar la columna "prob" del DataFrame malign_reduced_data
malign_reduced_data = malign_reduced_data.drop('prob', axis=1)

# Guardar los datos benignos en un archivo CSV
benign_data.to_csv('benign_data.csv', index=False)

# Guardar los datos malignos reducidos en un archivo CSV sin la columna "prob"
malign_reduced_data.to_csv('malign_reduced_data.csv', index=False)
"""

'\n# Concatenar todos los DataFrames con datos benignos en un solo DataFrame\nbenign_data = pd.concat([benign_samples])\n\n# Concatenar todos los DataFrames con datos malignos reducidos en un solo DataFrame\nmalign_reduced_data = pd.concat(malign_samples_reduced)\n\n# Eliminar la columna "prob" del DataFrame malign_reduced_data\nmalign_reduced_data = malign_reduced_data.drop(\'prob\', axis=1)\n\n# Guardar los datos benignos en un archivo CSV\nbenign_data.to_csv(\'benign_data.csv\', index=False)\n\n# Guardar los datos malignos reducidos en un archivo CSV sin la columna "prob"\nmalign_reduced_data.to_csv(\'malign_reduced_data.csv\', index=False)\n'

División entre Train, Test y Validación

In [8]:
"""
from sklearn.model_selection import train_test_split

# Concatenar los datos benignos y malignos en un solo DataFrame
all_data = pd.concat([benign_data, malign_reduced_data])

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
all_train, all_test = train_test_split(all_data, test_size=0.3, random_state=42)

# Dividir el conjunto de prueba en conjuntos de prueba y validación (20% prueba, 10% validación)
all_test, all_val = train_test_split(all_test, test_size=0.5, random_state=42)

# Separar los datos benignos y malignos en los conjuntos correspondientes
benign_train = all_train[all_train.iloc[:, 77] == 'BENIGN']
malign_train = all_train[all_train.iloc[:, 77] != 'BENIGN']
benign_test = all_test[all_test.iloc[:, 77] == 'BENIGN']
malign_test = all_test[all_test.iloc[:, 77] != 'BENIGN']
benign_val = all_val[all_val.iloc[:, 77] == 'BENIGN']
malign_val = all_val[all_val.iloc[:, 77] != 'BENIGN']

# Guardar los conjuntos de entrenamiento en archivos CSV
pd.concat([benign_train, malign_train]).to_csv('train.csv', index=False)

# Guardar los conjuntos de prueba en archivos CSV
pd.concat([benign_test, malign_test]).to_csv('test.csv', index=False)

# Guardar los conjuntos de validación en archivos CSV
pd.concat([benign_val, malign_val]).to_csv('validation.csv', index=False)

"""

"\nfrom sklearn.model_selection import train_test_split\n\n# Concatenar los datos benignos y malignos en un solo DataFrame\nall_data = pd.concat([benign_data, malign_reduced_data])\n\n# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)\nall_train, all_test = train_test_split(all_data, test_size=0.3, random_state=42)\n\n# Dividir el conjunto de prueba en conjuntos de prueba y validación (20% prueba, 10% validación)\nall_test, all_val = train_test_split(all_test, test_size=0.5, random_state=42)\n\n# Separar los datos benignos y malignos en los conjuntos correspondientes\nbenign_train = all_train[all_train.iloc[:, 77] == 'BENIGN']\nmalign_train = all_train[all_train.iloc[:, 77] != 'BENIGN']\nbenign_test = all_test[all_test.iloc[:, 77] == 'BENIGN']\nmalign_test = all_test[all_test.iloc[:, 77] != 'BENIGN']\nbenign_val = all_val[all_val.iloc[:, 77] == 'BENIGN']\nmalign_val = all_val[all_val.iloc[:, 77] != 'BENIGN']\n\n# Guardar los conjuntos de entrenam